In [1]:
!pip install pyxdf
!pip install mne
!pip install PyQt5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 58.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.8/361.8 KB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 MB 13.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
import pyxdf
import pandas as pd
import mne
import PyQt5
import numpy as np
from scipy.integrate import simps
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
import glob
import os

ImportError: ignored

In [ ]:
def read_data(fname):
    streams,fileheader= pyxdf.load_xdf(fname)
    stream = streams[1]
    nb_chan = int(stream['info']['channel_count'][0])
    if (nb_chan==5):
      channels = [(stream['info']['desc'][0]['channels'][0]['channel'][i]['label'][0]) for i in range(nb_chan)]
      df = pd.DataFrame(stream['time_series'], stream['time_stamps'], channels)

      d = df.to_numpy()
      d = np.transpose(d)
      data = d
      return data
    else:
      streamr = streams[0]
      nb_chans = int(streamr['info']['channel_count'][0])
      channels = [(streamr['info']['desc'][0]['channels'][0]['channel'][i]['label'][0]) for i in range(nb_chans)]
      df = pd.DataFrame(streamr['time_series'], streamr['time_stamps'], channels)

      d = df.to_numpy()
      d = np.transpose(d)
      data = d
      return data
def bandpower_from_psd(
    psd,
    freqs,
    ch_names=None,
    bands=[
        (0.5, 4, "Delta"),
        (4, 8, "Theta"),
        (8, 12, "Alpha"),
        (12, 16, "Sigma"),
        (16, 30, "Beta"),
        (30, 40, "Gamma"),
    ],
    relative=True,
):

    # Type checks
    assert isinstance(bands, list), "bands must be a list of tuple(s)"
    assert isinstance(relative, bool), "relative must be a boolean"

    # Safety checks
    freqs = np.asarray(freqs)
    assert freqs.ndim == 1
    psd = np.atleast_2d(psd)
    assert psd.ndim == 2, "PSD must be of shape (n_channels, n_freqs)."
    all_freqs = np.hstack([[b[0], b[1]] for b in bands])
    fmin, fmax = min(all_freqs), max(all_freqs)
    idx_good_freq = np.logical_and(freqs >= fmin, freqs <= fmax)
    freqs = freqs[idx_good_freq]
    res = freqs[1] - freqs[0]
    nchan = psd.shape[0]
    assert nchan < psd.shape[1], "PSD must be of shape (n_channels, n_freqs)."
    if ch_names is not None:
        ch_names = np.atleast_1d(np.asarray(ch_names, dtype=str))
        assert ch_names.ndim == 1, "ch_names must be 1D."
        assert len(ch_names) == nchan, "ch_names must match psd.shape[0]."
    else:
        ch_names = ["CHAN" + str(i).zfill(3) for i in range(nchan)]
    bp = np.zeros((nchan, len(bands)), dtype=np.float64)
    psd = psd[:, idx_good_freq]
    total_power = simps(psd, dx=res)
    total_power = total_power[..., np.newaxis]

    # Check if there are negative values in PSD
    if (psd < 0).any():
        msg = (
            "There are negative values in PSD. This will result in incorrect "
            "bandpower values. We highly recommend working with an "
            "all-positive PSD. For more details, please refer to: "
            "https://github.com/raphaelvallat/yasa/issues/29"
        )
        logger.warning(msg)

    # Enumerate over the frequency bands
    labels = []
    for i, band in enumerate(bands):
        b0, b1, la = band
        labels.append(la)
        idx_band = np.logical_and(freqs >= b0, freqs <= b1)
        bp[:, i] = simps(psd[:, idx_band], dx=res)

    if relative:
        bp /= total_power

    # Convert to DataFrame
    bp = pd.DataFrame(bp, columns=labels)
    bp["TotalAbsPow"] = np.squeeze(total_power)
    bp["FreqRes"] = res
    # bp['WindowSec'] = 1 / res
    bp["Relative"] = relative
    bp["Chan"] = ch_names
    bp = bp.set_index("Chan").reset_index()
    # Add hidden attributes
    bp.bands_ = str(bands)
    return bp,psd

data = read_data('/content/drive/MyDrive/Data Vr + Meditation/B22011/EEG/ses-S001_3_2/eeg/sub-P001_011_ses-S001_3_2_task-Default_run-001_eeg.xdf')
from scipy import signal
win = int(4*256)
relative = True
# Calculate the PSD over the whole data
freqs, psd = signal.welch(data, 256, nperseg=win)
bp = bandpower_from_psd(psd, freqs, ['TP9', 'AF7', 'AF8', 'TP10', 'Right AUX'], relative=False)
df=pd.DataFrame(bp[0])
print(df)

        Chan        Delta       Theta       Alpha      Sigma       Beta  \
0        TP9   476.474579  219.874969  172.162323  48.231243  34.850868   
1        AF7   977.950134   21.744122    6.341209   2.960512   5.613937   
2        AF8  2474.195312   74.864037   15.174440   6.487123   8.680363   
3       TP10   525.163025   35.662437   32.000023   6.598264  19.875916   
4  Right AUX     0.000000    0.000000    0.000000   0.000000   0.000000   

       Gamma  TotalAbsPow  FreqRes  Relative  
0  31.243622   982.837646     0.25     False  
1   3.661045  1018.270874     0.25     False  
2   5.729907  2585.131348     0.25     False  
3  11.572397   630.871948     0.25     False  
4   0.000000     0.000000     0.25     False  


In [ ]:
path = "/content/drive/MyDrive/Data Vr + Meditation/xdf"
xdf_files = glob.glob(path + "/*.xdf")
name=[]
list1=[]
list2=[]
list3=[]
list4=[]
list5=[]
list6=[]
list7=[]
list8=[]
list9=[]
list10=[]
list11=[]
list12=[]
list13=[]
list14=[]
list15=[]
list16=[]
list17=[]
list18=[]
list19=[]
list20=[]
list21=[]
list22=[]
list23=[]
list24=[]
list25=[]
list26=[]
list27=[]
list28=[]
for j in xdf_files:
    file_name = os.path.basename(j)
    pth=(os.path.splitext(file_name)[0])
    name.append(pth[9:13]+pth[22:25])
    def read_data(fname):
        streams,fileheader=pyxdf.load_xdf(fname)
        stream = streams[1]
        nb_chan = int(stream['info']['channel_count'][0])
        if (nb_chan==5):
          channels = [(stream['info']['desc'][0]['channels'][0]['channel'][i]['label'][0]) for i in range(nb_chan)]
          df = pd.DataFrame(stream['time_series'], stream['time_stamps'], channels)

          d = df.to_numpy()
          d = np.transpose(d)
          data = d
          return data
        else:
          streamr = streams[0]
          nb_chans = int(streamr['info']['channel_count'][0])
          channels = [(streamr['info']['desc'][0]['channels'][0]['channel'][i]['label'][0]) for i in range(nb_chans)]
          df = pd.DataFrame(streamr['time_series'], streamr['time_stamps'], channels)

          d = df.to_numpy()
          d = np.transpose(d)
          data = d
          return data
    def bandpower_from_psd(
        psd,
        freqs,
        ch_names=None,
        bands=[
            (0.5, 4, "Delta"),
            (4, 8, "Theta"),
            (8, 12, "Alpha"),
            (12, 16, "Sigma"),
            (16, 30, "Beta"),
            (30, 40, "Gamma"),
        ],
        relative=True,
    ):

        # Type checks
        assert isinstance(bands, list), "bands must be a list of tuple(s)"
        assert isinstance(relative, bool), "relative must be a boolean"

        # Safety checks
        freqs = np.asarray(freqs)
        assert freqs.ndim == 1
        psd = np.atleast_2d(psd)
        assert psd.ndim == 2, "PSD must be of shape (n_channels, n_freqs)."
        all_freqs = np.hstack([[b[0], b[1]] for b in bands])
        fmin, fmax = min(all_freqs), max(all_freqs)
        idx_good_freq = np.logical_and(freqs >= fmin, freqs <= fmax)
        freqs = freqs[idx_good_freq]
        res = freqs[1] - freqs[0]
        nchan = psd.shape[0]
        assert nchan < psd.shape[1], "PSD must be of shape (n_channels, n_freqs)."
        if ch_names is not None:
            ch_names = np.atleast_1d(np.asarray(ch_names, dtype=str))
            assert ch_names.ndim == 1, "ch_names must be 1D."
            assert len(ch_names) == nchan, "ch_names must match psd.shape[0]."
        else:
            ch_names = ["CHAN" + str(i).zfill(3) for i in range(nchan)]
        bp = np.zeros((nchan, len(bands)), dtype=np.float64)
        psd = psd[:, idx_good_freq]
        total_power = simps(psd, dx=res)
        total_power = total_power[..., np.newaxis]

        # Check if there are negative values in PSD
        if (psd < 0).any():
            msg = (
                "There are negative values in PSD. This will result in incorrect "
                "bandpower values. We highly recommend working with an "
                "all-positive PSD. For more details, please refer to: "
                "https://github.com/raphaelvallat/yasa/issues/29"
            )
            logger.warning(msg)

        # Enumerate over the frequency bands
        labels = []
        for i, band in enumerate(bands):
            b0, b1, la = band
            labels.append(la)
            idx_band = np.logical_and(freqs >= b0, freqs <= b1)
            bp[:, i] = simps(psd[:, idx_band], dx=res)

        if relative:
            bp /= total_power

        # Convert to DataFrame
        bp = pd.DataFrame(bp, columns=labels)
        bp["TotalAbsPow"] = np.squeeze(total_power)
        bp["FreqRes"] = res
        # bp['WindowSec'] = 1 / res
        bp["Relative"] = relative
        bp["Chan"] = ch_names
        bp = bp.set_index("Chan").reset_index()
        # Add hidden attributes
        bp.bands_ = str(bands)
        return bp,psd

    data = read_data(j)
    from scipy import signal
    win = int(4*256)
    relative = True
    # Calculate the PSD over the whole data
    freqs, psd = signal.welch(data, 256, nperseg=win)
    bp = bandpower_from_psd(psd, freqs, ch_names = ['TP9', 'AF7', 'AF8', 'TP10', 'Right AUX'], relative=False)
    df=pd.DataFrame(bp[0])
    
    # APPENDING INTO LISTS 
    list1.append(df['Delta'][0])
    list2.append(df['Delta'][1])
    list3.append(df['Delta'][2])
    list4.append(df['Delta'][3])
    list5.append(df['Theta'][0])
    list6.append(df['Theta'][1])
    list7.append(df['Theta'][2])
    list8.append(df['Theta'][3])
    list9.append(df['Alpha'][0])
    list10.append(df['Alpha'][1])
    list11.append(df['Alpha'][2])
    list12.append(df['Alpha'][3])
    list13.append(df['Sigma'][0])
    list14.append(df['Sigma'][1])
    list15.append(df['Sigma'][2])
    list16.append(df['Sigma'][3])
    list17.append(df['Beta'][0])
    list18.append(df['Beta'][1])
    list19.append(df['Beta'][2])
    list20.append(df['Beta'][3])
    list21.append(df['Gamma'][0])
    list22.append(df['Gamma'][1])
    list23.append(df['Gamma'][2])
    list24.append(df['Gamma'][3])
    list25.append(df['TotalAbsPow'][0])
    list26.append(df['TotalAbsPow'][1])
    list27.append(df['TotalAbsPow'][2])
    list28.append(df['TotalAbsPow'][3])

In [ ]:
# CONVERT TO DATAFRAME
csv1=pd.DataFrame(list1)
csv2=pd.DataFrame(list2)
csv3=pd.DataFrame(list3)
csv4=pd.DataFrame(list4)
csv5=pd.DataFrame(list5)
csv6=pd.DataFrame(list6)
csv7=pd.DataFrame(list7)
csv8=pd.DataFrame(list8)
csv9=pd.DataFrame(list9)
csv10=pd.DataFrame(list10)
csv11=pd.DataFrame(list11)
csv12=pd.DataFrame(list12)
csv13=pd.DataFrame(list13)
csv14=pd.DataFrame(list14)
csv15=pd.DataFrame(list15)
csv16=pd.DataFrame(list16)
csv17=pd.DataFrame(list17)
csv18=pd.DataFrame(list18)
csv19=pd.DataFrame(list19)
csv20=pd.DataFrame(list20)
csv21=pd.DataFrame(list21)
csv22=pd.DataFrame(list22)
csv23=pd.DataFrame(list23)
csv24=pd.DataFrame(list24)
csv25=pd.DataFrame(list25)
csv26=pd.DataFrame(list26)
csv27=pd.DataFrame(list27)
csv28=pd.DataFrame(list28)
comb= pd.concat([csv1,csv2,csv3,csv4,csv5,csv6,csv7,csv8,csv9,csv10,csv11,csv12,csv13,csv14,csv15,csv16,csv17,csv18,csv19,csv20,csv21,csv22,csv23,csv24,csv25,csv26,csv27,csv28], axis=1)
comb.columns =['TP9_Delta','AF7_Delta','AF8-Delta','TP10_Delta','TP9_Theta','AF7_Theta','AF8-Theta','TP10_Theta','TP9_Alpha','AF7_Alpha','AF8-Alpha','TP10_Alpha','TP9_Sigma','AF7_Sigma','AF8-Sigma','TP10_Sigma','TP9_Beta','AF7_Beta','AF8-Beta','TP10_Beta','TP9_Gamma','AF7_Gamma','AF8-Gamma','TP10_Gamma','TP9_TotalAbsPow','AF7_TotalAbsPow','AF8-TotalAbsPow','TP10_TotalAbsPow']
comb.index=name
comb.to_csv("Final EEG.csv")

In [ ]:
# T-TEST
# EEG
import scipy.stats as stats
data_group1 = comb.iloc[0,:].values
data_group2 = comb.iloc[1,:].values
stats.ttest_ind(a=data_group1, b=data_group2, equal_var=True)

In [ ]:
# N-BACK
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/Data Vr + Meditation/N_back_result.csv')
data_group1 = data.iloc[2,1:].values
data_group2 = data.iloc[4,1:].values
stats.ttest_ind(a=data_group1, b=data_group2, equal_var=True)

In [ ]:
# STROOP
import pandas as pd
DF = pd.read_csv('/content/drive/MyDrive/Data Vr + Meditation/Stroop_result.csv')
df2=DF.drop(['Time'], axis=1)
df4=(df2.iloc[:,0].values)[7:]